In [30]:
from collections import defaultdict

import yfinance as yf
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from finta import TA
import numpy as np
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import jaccard_score
import seaborn as sn
from tabulate import tabulate
from xgboost import XGBClassifier
from ta import add_all_ta_features
from sklearn.feature_selection import RFE
import xgboost as xgb
import ast

In [31]:
data = pd.read_csv(
    'C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\nfold\\result_train_points_AAPL_8_13_05_2021 15_56_38.csv', index_col=0)

data

,0_x,0_y,2_x,3_x,4_x,5_x,6_x,7_x,8_x,9_x,...,60_y,61_y,62_y,63_y,64_y,65_y,66_y,67_y,68_y,69_y
0,DecisionTreeClassifier 1,"(4, 3, 0, 5, 8, 4, 17)","(1, 3, 0, 9, 3, 5, 17)","(5, 3, 0, 5, 3, 9, 17)","(3, 0, 0, 17, 0, 0, 17)","(4, 2, 0, 11, 5, 1, 17)","(0, 5, 3, 1, 10, 6, 17)","(2, 0, 3, 3, 11, 3, 17)","(8, 3, 0, 8, 4, 5, 17)","(2, 7, 2, 3, 9, 5, 17)",...,"(751, 770, 747, 913, 893, 914, 2720)","(804, 808, 853, 912, 907, 918, 2737)","(808, 833, 852, 918, 914, 922, 2754)","(815, 788, 756, 936, 909, 926, 2771)","(795, 800, 767, 938, 910, 940, 2788)","(837, 823, 822, 939, 932, 934, 2805)","(828, 796, 777, 951, 932, 939, 2822)","(815, 765, 860, 958, 931, 950, 2839)","(796, 752, 878, 952, 943, 961, 2856)","(782, 800, 840, 959, 960, 954, 2873)"
1,DecisionTreeClassifier 2,"(4, 3, 2, 5, 8, 4, 17)","(2, 3, 1, 9, 3, 5, 17)","(4, 3, 0, 5, 3, 9, 17)","(3, 0, 0, 17, 0, 0, 17)","(4, 2, 0, 11, 5, 1, 17)","(0, 6, 0, 1, 10, 6, 17)","(2, 0, 3, 3, 11, 3, 17)","(6, 3, 1, 8, 4, 5, 17)","(1, 7, 2, 3, 9, 5, 17)",...,"(908, 880, 913, 913, 893, 914, 2720)","(912, 907, 918, 912, 907, 918, 2737)","(918, 914, 922, 918, 914, 922, 2754)","(931, 896, 925, 936, 909, 926, 2771)","(938, 910, 940, 938, 910, 940, 2788)","(934, 919, 933, 939, 932, 934, 2805)","(946, 919, 938, 951, 932, 939, 2822)","(958, 930, 950, 958, 931, 950, 2839)","(944, 899, 960, 952, 943, 961, 2856)","(956, 960, 954, 959, 960, 954, 2873)"
2,DecisionTreeClassifier 3,"(5, 2, 1, 5, 8, 4, 17)","(2, 3, 0, 9, 3, 5, 17)","(4, 3, 9, 5, 3, 9, 17)","(3, 0, 0, 17, 0, 0, 17)","(11, 0, 0, 11, 5, 1, 17)","(1, 5, 0, 1, 10, 6, 17)","(0, 0, 3, 3, 11, 3, 17)","(3, 4, 0, 8, 4, 5, 17)","(0, 3, 3, 3, 9, 5, 17)",...,"(698, 634, 666, 913, 893, 914, 2720)","(628, 695, 644, 912, 907, 918, 2737)","(707, 594, 633, 918, 914, 922, 2754)","(721, 683, 616, 936, 909, 926, 2771)","(725, 794, 679, 938, 910, 940, 2788)","(769, 762, 709, 939, 932, 934, 2805)","(750, 725, 736, 951, 932, 939, 2822)","(687, 730, 594, 958, 931, 950, 2839)","(716, 795, 685, 952, 943, 961, 2856)","(764, 789, 687, 959, 960, 954, 2873)"
3,DecisionTreeClassifier 4,"(3, 6, 0, 5, 8, 4, 17)","(4, 0, 2, 9, 3, 5, 17)","(0, 3, 4, 5, 3, 9, 17)","(4, 0, 0, 17, 0, 0, 17)","(6, 1, 0, 11, 5, 1, 17)","(1, 4, 0, 1, 10, 6, 17)","(3, 1, 3, 3, 11, 3, 17)","(4, 0, 1, 8, 4, 5, 17)","(2, 7, 0, 3, 9, 5, 17)",...,"(777, 783, 839, 913, 893, 914, 2720)","(763, 833, 809, 912, 907, 918, 2737)","(772, 858, 794, 918, 914, 922, 2754)","(839, 842, 753, 936, 909, 926, 2771)","(806, 783, 696, 938, 910, 940, 2788)","(782, 805, 687, 939, 932, 934, 2805)","(863, 830, 740, 951, 932, 939, 2822)","(808, 768, 738, 958, 931, 950, 2839)","(793, 798, 721, 952, 943, 961, 2856)","(768, 820, 711, 959, 960, 954, 2873)"
4,DecisionTreeClassifier 5,"(3, 3, 0, 5, 8, 4, 17)","(4, 3, 2, 9, 3, 5, 17)","(0, 3, 4, 5, 3, 9, 17)","(6, 0, 0, 17, 0, 0, 17)","(5, 1, 0, 11, 5, 1, 17)","(1, 4, 0, 1, 10, 6, 17)","(2, 7, 3, 3, 11, 3, 17)","(7, 1, 0, 8, 4, 5, 17)","(1, 7, 0, 3, 9, 5, 17)",...,"(906, 892, 899, 913, 893, 914, 2720)","(904, 899, 913, 912, 907, 918, 2737)","(917, 909, 919, 918, 914, 922, 2754)","(931, 901, 915, 936, 909, 926, 2771)","(918, 896, 919, 938, 910, 940, 2788)","(930, 918, 909, 939, 932, 934, 2805)","(934, 917, 927, 951, 932, 939, 2822)","(939, 922, 936, 958, 931, 950, 2839)","(940, 937, 935, 952, 943, 961, 2856)","(954, 949, 945, 959, 960, 954, 2873)"
5,RandomForestClassifier 4,"(5, 4, 0, 5, 8, 4, 17)","(8, 3, 2, 9, 3, 5, 17)","(4, 3, 1, 5, 3, 9, 17)","(5, 0, 0, 17, 0, 0, 17)","(11, 0, 0, 11, 5, 1, 17)","(0, 8, 0, 1, 10, 6, 17)","(2, 3, 0, 3, 11, 3, 17)","(1, 0, 0, 8, 4, 5, 17)","(1, 5, 0, 3, 9, 5, 17)",...,"(579, 540, 505, 913, 893, 914, 2720)","(544, 586, 466, 912, 907, 918, 2737)","(548, 588, 467, 918, 914, 922, 2754)","(600, 537, 495, 936, 909, 926, 2771)","(595, 525, 510, 938, 910, 940, 2788)","(580, 589, 467, 939, 932, 934, 2805)","(610, 562, 477, 951, 932, 939, 2822)","(617, 522, 525, 958, 931, 950, 2839)","(589, 545, 532, 952, 943, 961, 2856)","(583, 619, 463, 959, 960, 954, 2873)"
6,R

In [32]:

df_copy_inc = data.copy()
df_copy_inc_all = data.copy()

def only_indx(x, indx):
    if"(" not in x:
        return x
    literal_eval = ast.literal_eval(x)
    return literal_eval[indx]


df_copy_inc = df_copy_inc.applymap(lambda x:only_indx(x,0))

In [33]:
df_copy_inc.to_csv('C:\\Users\\exomat\\Desktop\\repo\\magisterka_analiza\\data\\results\\nfold\\result_train_points__cor_AAPL_8_13_05_2021 15_56_38.csv')